In [143]:
import pandas as pd
import numpy as np
import re

In [144]:
df = pd.read_csv("flight_data_kiwi_03_02_25.csv")

In [145]:
# First fill Price from Back Company if needed
df['Price'] = df['Price'].fillna(df['Back Company'])

# Then fill Back Company from Dep Company if needed
df['Back Company'] = df['Back Company'].fillna(df['Dep Company'])

# Optionally clear out Back Company in rows where it was used for Price
mask_used_for_price = df['Price'] == df['Back Company']
df.loc[mask_used_for_price, 'Back Company'] = np.nan

In [146]:
df['Back Company'] = df['Back Company'].fillna(df['Dep Company'])

In [147]:
# 1) Make sure 'Stops.1' is all strings
df['Stops.1'] = df['Stops.1'].fillna('').astype(str)

# 2) Create new columns
df['Stops'] = np.nan
df['Stops City'] = np.nan

# 3) Mask for direct flights
mask_direct = df['Stops.1'].str.lower().eq('direct')
df.loc[mask_direct, 'Stops'] = 0
df.loc[mask_direct, 'Stops City'] = np.nan

# 4) Mask for flights that contain "stop" or "stops"
mask_stops = df['Stops.1'].str.lower().str.contains('stop', na=False)

# (a) Extract the number of stops
df.loc[mask_stops, 'Stops'] = (
    df.loc[mask_stops, 'Stops.1']
      .str.extract(r'(\d+)')[0]  # Captures digits before "stop"
      .astype(float)             # or int if always whole numbers
)

In [148]:
# Remove the leading "<digits> stop(s) • " part from Stops.1
df['Stops City'] = df['Stops.1'].replace(
    to_replace=r'^\d+\s*stops?\s*•\s*',   # e.g. "1 stop •", "2 stops •"
    value='',
    regex=True
)

# Then set "Direct" rows to NaN in the new column
df.loc[df['Stops.1'] == 'Direct', 'Stops City'] = np.nan

In [149]:
df.to_csv("flight_data_kiwi_clean.csv", index=False)